In [477]:
# !pip install plotly

In [478]:
import plotly
plotly.__version__

'2.0.12'

In [574]:
import pickle 
import re
def get_all_emails_with_time_of_repository(repoId):
    all_emails = []
    for page_pickle in range(1,10):
        try:
            articles = pickle.load(open('../data/all_articles_on'+str(repoId)+'_'+str(page_pickle)+'.pkl','rb'))
            for page in range(0,len(articles)):
                for item in articles[page]['data']:
                    text = item['fullText']
                    article_id=item['id']
                    year=item['year']
                    match = re.findall(r'[\w\.-]+@[\w\.-]+', text)
#                     print(match)
                    all_emails.append([article_id, year,match])
        except (OSError,IOError) as e:
            print(e)
    return all_emails

In [575]:
#this takes some time to load - loads ~1.5G of data
emails_of_repository=dict()
# cached_repos=[1,86,39,136,48]
cached_repos=[1,86]
for i in cached_repos:
    emails_of_repository[i]=get_all_emails_with_time_of_repository(i)

In [576]:
emails_of_repository[1][0:10]

[[u'19787393',
  2014,
  [u'L.V.Burgin@leeds.ac.uk',
   u'Lorna.ramsay@education.gsi.gov.uk',
   u'r.aspden@abdn.ac.uk']],
 [u'19787406', 2014, [u'david.mcghee@abdn.ac.uk']],
 [u'20327925', 2014, [u'j.mercer@abdn.ac.uk', u'michelle.murphy@abdn.ac.uk']],
 [u'20664118', 2014, [u'Sarah.Harris@igmm.ed.ac.uk.']],
 [u'20664131', 2014, [u'T.Gillingwater@ed.ac.uk.']],
 [u'20664054', 2014, [u'jiabao.he@abdn.ac.uk']],
 [u'20664371', 2014, []],
 [u'19888921', 2014, [u's.schwarzkopf@gmail.com']],
 [u'20664053',
  2014,
  [u'marie.castellazzi@hutton.ac.uk',
   u'diana.feliciano@abdn.ac.uk',
   u'iain.brown@hutton.ac.uk']],
 [u'20664046', 2014, [u'Jill.Francis.1@city.ac.uk']]]

In [578]:
def flatten_list(unevenList):
    flat_list=[]
    for sublist in unevenList:
        aid=sublist[0]
        y=sublist[1]
        emails = sublist[2]
        for item in emails:
            flat_list.append([aid,y,item])
    return flat_list

In [579]:
# cached_repos={1:'abdn.ac.uk',86:'open.ac.uk',39:'ed.ac.uk',136:'warwick.ac.uk',48:'gla.ac.uk'}
cached_repos={1:'abdn.ac.uk',86:'open.ac.uk'}

for i in cached_repos:
    emails_of_repository[i]=flatten_list(emails_of_repository[i])

In [581]:
emails_of_repository[1][0:10]

[[u'19787393', 2014, u'L.V.Burgin@leeds.ac.uk'],
 [u'19787393', 2014, u'Lorna.ramsay@education.gsi.gov.uk'],
 [u'19787393', 2014, u'r.aspden@abdn.ac.uk'],
 [u'19787406', 2014, u'david.mcghee@abdn.ac.uk'],
 [u'20327925', 2014, u'j.mercer@abdn.ac.uk'],
 [u'20327925', 2014, u'michelle.murphy@abdn.ac.uk'],
 [u'20664118', 2014, u'Sarah.Harris@igmm.ed.ac.uk.'],
 [u'20664131', 2014, u'T.Gillingwater@ed.ac.uk.'],
 [u'20664054', 2014, u'jiabao.he@abdn.ac.uk'],
 [u'19888921', 2014, u's.schwarzkopf@gmail.com']]

In [582]:
import pandas as pd
repos_collabs_df = pd.DataFrame()
for i in cached_repos:
    repo_df=pd.DataFrame.from_records(emails_of_repository[i])
    repo_df['repository_domain']=cached_repos[i]
    repos_collabs_df = repos_collabs_df.append(repo_df)
repos_collabs_df.head()

,0,1,2,repository_domain
0,19787393,2014,L.V.Burgin@leeds.ac.uk,abdn.ac.uk
1,19787393,2014,Lorna.ramsay@education.gsi.gov.uk,abdn.ac.uk
2,19787393,2014,r.aspden@abdn.ac.uk,abdn.ac.uk
3,19787406,2014,david.mcghee@abdn.ac.uk,abdn.ac.uk
4,20327925,2014,j.mercer@abdn.ac.uk,abdn.ac.uk


In [583]:
len(repos_collabs_df)

5235

In [584]:
def extract_domain(email):
    return email.split("@")[1].rstrip(",").rstrip(".")
repos_collabs_df['author_domain']=repos_collabs_df.apply(lambda row: extract_domain(row[2]),axis=1)
repos_collabs_df.head()

,0,1,2,repository_domain,author_domain
0,19787393,2014,L.V.Burgin@leeds.ac.uk,abdn.ac.uk,leeds.ac.uk
1,19787393,2014,Lorna.ramsay@education.gsi.gov.uk,abdn.ac.uk,education.gsi.gov.uk
2,19787393,2014,r.aspden@abdn.ac.uk,abdn.ac.uk,abdn.ac.uk
3,19787406,2014,david.mcghee@abdn.ac.uk,abdn.ac.uk,abdn.ac.uk
4,20327925,2014,j.mercer@abdn.ac.uk,abdn.ac.uk,abdn.ac.uk


In [585]:
#filter domains with too short length or too long
mask = (repos_collabs_df['author_domain'].str.len() >5 )

repos_collabs_filtered_df = repos_collabs_df.loc[mask]
mask2 = (repos_collabs_filtered_df['author_domain'].str.len() <25 )
repos_collabs_filtered_df = repos_collabs_filtered_df.loc[mask2]
len(repos_collabs_filtered_df)



4948

In [586]:
len(repos_collabs_df)

5235

In [587]:
repos_collabs_grouped = pd.DataFrame(repos_collabs_filtered_df.groupby(['author_domain','repository_domain']).size().rename('counts').reset_index(name='counts').sort_values(['counts'],ascending=False))
repos_collabs_grouped.head(10)

,author_domain,repository_domain,counts
31,abdn.ac.uk,abdn.ac.uk,1184
769,open.ac.uk,open.ac.uk,891
706,ncl.ac.uk,abdn.ac.uk,67
369,gmail.com,abdn.ac.uk,66
16,Future,open.ac.uk,65
292,ed.ac.uk,abdn.ac.uk,55
726,nhs.net,abdn.ac.uk,44
370,gmail.com,open.ac.uk,37
402,hkucc.hku.hk,abdn.ac.uk,35
744,nottingham.ac.uk,open.ac.uk,30


In [588]:
source_repos_grouped = pd.DataFrame(repos_collabs_filtered_df.groupby('repository_domain').size().rename('counts').reset_index(name='counts').sort_values(['counts'],ascending=False))
source_repos_grouped.head()
# len(repos_collabs_grouped['author_domain'])
# repos_collabs_grouped.set_index('counts', inplace=True)
# repos_collabs_grouped.head(10)

,repository_domain,counts
0,abdn.ac.uk,2823
1,open.ac.uk,2125


In [589]:
# repos_collabs_grouped['author_domain'].unique()[:100]

In [590]:
# repos_collabs_grouped.set_index('author_domain', inplace=True)
# repos_collabs_grouped.head()

In [591]:
source_repos_list = source_repos_grouped['repository_domain'].tolist()

In [592]:
source_repos_list.index('warwick.ac.uk')

ValueError: 'warwick.ac.uk' is not in list

In [644]:
# build the nodes of the sankey diagram
sankey_labels = repos_collabs_grouped['author_domain'].tolist()
sankey_labels.extend(source_repos_list)

#make sure the last 5 are the repository ones (included once again)
sankey_labels[:5]

[u'abdn.ac.uk', u'open.ac.uk', u'ncl.ac.uk', u'gmail.com', u'Future']

In [645]:
#check that the last elements are the domain of source repos
sankey_labels[-10:]

[u'inf.ed.ac.uk',
 u'info.fundp.ac.be',
 u'iol.ie',
 u'iol.ie',
 u'ioz.ac.cn',
 u'ipdglobal.com',
 u'ipfran.ru',
 u'zsl.org',
 'abdn.ac.uk',
 'open.ac.uk']

In [646]:
def find_index_of_first_occurence_from_tail(haystack,value):
    try:
        i =len(haystack) - 1 - haystack[::-1].index(value)
        return i
    except:
        print("EXCEPTION!!!",haystack[0:5],value)
        return 

In [647]:
find_index_of_first_occurence_from_tail(sankey_labels,'gla.ac.uk')

('EXCEPTION!!!', [u'abdn.ac.uk', u'open.ac.uk', u'ncl.ac.uk', u'gmail.com', u'Future'], 'gla.ac.uk')


In [657]:
sankey_labels = sankey_labels[0:75]+sankey_labels[-2:]
def break_frame_columns_to_sankey_lists(frame):
    sankey_source=[]
    sankey_target=[]
    sankey_values=[]
    s_labels=[]
    for index, row in frame.iterrows():
        sankey_source.append(sankey_labels.index(row['author_domain']))
#         print(row['repository_domain'])
        targetRepoIndex = find_index_of_first_occurence_from_tail(sankey_labels,row['repository_domain'])
        sankey_target.append(targetRepoIndex)#we are looking for the last item in the list of nodes
        if (row['author_domain']==row['repository_domain']):
            sankey_values.append(row['counts']/3)#divide by N so that the basic flow does not dominate the chart
        else:
            sankey_values.append(row['counts'])
        s_labels.append("{} collaborators from {} to {}".format(row['counts'],row['author_domain'],row['repository_domain']))
    return sankey_source,sankey_target,sankey_values,s_labels
              
# find_index_of_first_occurence_from_tail(sankey_labels,'warwick.ac.uk')
s_source,s_target,s_values,s_labels=break_frame_columns_to_sankey_lists(repos_collabs_grouped[0:75])


ValueError: u'soton.ac.uk' is not in list

In [658]:
#let's view what is in the nodes/links
sankey_labels[0:10]
# sankey_labels[-10:]

[u'abdn.ac.uk',
 u'open.ac.uk',
 u'ncl.ac.uk',
 u'gmail.com',
 u'Future',
 u'ed.ac.uk',
 u'nhs.net',
 u'gmail.com',
 u'hkucc.hku.hk',
 u'nottingham.ac.uk']

In [659]:
print(len(s_source),len(s_target),len(s_values),len(sankey_labels))

(50, 50, 50, 54)


In [660]:
#sankey colors of nodes - for now all the same - can be randomly selected
sankey_link_colors=[]
for i in range(0,len(s_source)):
    sankey_link_colors.append("rgba(0,0,96,0.2)")
sankey_node_colors=[]
for i in range(0,len(sankey_labels)):
    sankey_node_colors.append("rgba(96,96,0,0.5)")

In [661]:
# the lists of labels and colors should be equal in size
print(len(sankey_labels),len(sankey_node_colors))

(54, 54)


In [662]:
#last check - all lengths of the lists below should be equal
print(len(s_source),len(s_target),len(s_values),len(sankey_link_colors),len(s_labels))

(50, 50, 50, 50, 50)


In [663]:
print(s_source[-10:])

[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]


In [664]:
print(s_target[0:10])

[50, 51, 50, 50, 51, 50, 50, 51, 50, 51]


In [665]:
data_trace = dict(
    type='sankey',
    width = 1248,
    height = 1000,
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    orientation = "h",
    valueformat = ".0f",
    node = dict(
      pad = 2,
      thickness = 1,
      line = dict(
        color = "black",
        width = 0.5
      ),
      label =  sankey_labels,
      color =  sankey_node_colors
    ),
    link = dict(
      source =  s_source,
      target =  s_target,
      value =  s_values,
      color =  sankey_link_colors,
      label =  s_labels
  ))

layout =  dict(
    title = "Publication collaborations between Open university and Aberdeen university - top 50 collaborators displayed",
    font = dict(
      size = 10
    ),autosize=True
)

fig = dict(data=[data_trace], layout=layout)


plotly.offline.iplot(fig, validate=False)

In [666]:
# import plotly.plotly as py
# plotly.offline.iplot(fig, filename='2_unis_collab')